<a href="https://colab.research.google.com/github/SuriyaPriya17/Intelligent_system/blob/main/lstm_textGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import random
import tensorflow as tf
import requests
path = tf.keras.utils.get_file("shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

text = open(path, "r").read()
print("Text length:", len(text))
chars = sorted(list(set(text)))  # Unique characters
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}
n_chars = len(text)
n_vocab = len(chars)

seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

# Reshape X to [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# Normalize
X = X / float(n_vocab)
# One-hot encode y
y = np.zeros((n_patterns, n_vocab))
for i, pattern in enumerate(dataY):
    y[i, pattern] = 1

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X, y, epochs=8, batch_size=128)

def generate_text(model, seed, length=500, temperature=1.0):

    result = seed
    for _ in range(length):

        x = np.zeros((1, seq_length, 1))
        for t, char in enumerate(seed):
            x[0, t, 0] = char_to_int[char]
        x = x / float(n_vocab)

        prediction = model.predict(x, verbose=0)[0]

        prediction = np.log(prediction) / temperature
        prediction = np.exp(prediction) / np.sum(np.exp(prediction))

        index = np.random.choice(len(prediction), p=prediction)
        result += int_to_char[index]

        seed = seed[1:] + int_to_char[index]
    return result

seed = "To be, or not to be"
generated_text = generate_text(model, seed, length=200)
print("Generated Text:")
print(generated_text)
